In [1]:
#KÜTÜPHANELER

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

Veriler Fake ve True isminde iki farklı dosyada bulunuyor. Verileri okuyup iki farklı değişkene kaydettikten sonra iki dataframe de yeni label isminde sütun ekliyorum ve fake veriler için 0 true veriler için 1 olacak şekilde bu sütunu da dolduruyoruz. Daha sonra bu iki dataframe i birleştirip tek dataframe elde ediyoruz.

In [2]:
df_fake = pd.read_csv("Fake.csv")
df_true = pd.read_csv("True.csv")

df_fake["label"] = 0
df_true["label"] = 1

df = pd.concat([df_fake, df_true])

df = df.sample(frac=1, random_state = 4).reset_index(drop=True)

df.head()

,title,text,subject,date,label
0,Virginia Trump Supporter Patrols Polling Plac...,"A Trump supporter in Loudoun County, Virginia,...",News,"November 5, 2016",0
1,"Senate to act this week on Obamacare repeal, M...",WASHINGTON (Reuters) - The U.S. Senate will ta...,politicsNews,"January 8, 2017",1
2,Online ads offer legal option for U.S. electio...,SAN FRANCISCO (Reuters) - The laws that prohib...,politicsNews,"July 13, 2017",1
3,Hustler Owner Is So Fed Up With Trump That He...,Donald Trump is a danger to the nation and the...,News,"October 14, 2017",0
4,"Trump would freeze new federal regulations, re...",WASHINGTON (Reuters) - Republican presidential...,politicsNews,"August 8, 2016",1


In [3]:
#Eksik Veri Kontrolü

df.isnull().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

Modelde yalnızca text sütununu kullanarak tahmin yapmaya çalışacağız. Bu doğrultuda verinin içinde text ve label sütunlarını alıyoruz.

In [4]:
df_model = df[{"label", "text"}]

In [5]:
df_model

,label,text
0,0,"A Trump supporter in Loudoun County, Virginia,..."
1,1,WASHINGTON (Reuters) - The U.S. Senate will ta...
2,1,SAN FRANCISCO (Reuters) - The laws that prohib...
3,0,Donald Trump is a danger to the nation and the...
4,1,WASHINGTON (Reuters) - Republican presidential...
...,...,...
44893,0,21st Century Wire says This week s announcemen...
44894,0,
44895,1,(Reuters) - Montana Republican congressional c...
44896,0,A man ranting about Nazi Germany had a strange...


### Veri Ön İşleme

In [6]:
## buyuk-kucuk donusumu
df_model['text'] = df_model['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
#noktalama işaretleri
df_model['text'] = df_model['text'].str.replace('[^\w\s]','')
#sayılar
df_model['text'] = df_model['text'].str.replace('\d','')
#stopwords
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
sw = stopwords.words('english')
df_model['text'] = df_model['text'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
#seyreklerin silinmesi
sil = pd.Series(' '.join(df_model['text']).split()).value_counts()[-1000:]
df_model['text'] = df_model['text'].apply(lambda x: " ".join(x for x in x.split() if x not in sil))
#lemmi
from textblob import Word
#nltk.download('wordnet')
df_model['text'] = df_model['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

#KAYNAK: https://www.udemy.com/course/python-ile-makine-ogrenmesi/

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [7]:
df_model

,label,text
0,0,trump supporter loudoun county virginia spent ...
1,1,washington reuters u senate take first step to...
2,1,san francisco reuters law prohibit foreign nat...
3,0,donald trump danger nation world even outgoing...
4,1,washington reuters republican presidential can...
...,...,...
44893,0,st century wire say week announcement cia some...
44894,0,
44895,1,reuters montana republican congressional candi...
44896,0,man ranting nazi germany strange encounter msn...


In [8]:
X = df_model["text"]
y = df_model["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Count Vectors

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
count_vectorizer.fit(X_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [10]:
X_train_count = count_vectorizer.transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

#  TF-IDF

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf_vectorizer = TfidfVectorizer()
tf_idf_vectorizer.fit(X_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [12]:
X_train_tf_idf = tf_idf_vectorizer.transform(X_train)
X_test_tf_idf = tf_idf_vectorizer.transform(X_test)

In [13]:
from sklearn import metrics

# Decission Tree

### Count Vectors

In [14]:
from sklearn.tree import DecisionTreeClassifier

#Karar Ağacı Objesi
dectree = DecisionTreeClassifier()

#Modeli Eğitme
dectree_model_count = dectree.fit(X_train_count,y_train)

#Tahmin
dectree_y_pred_count = dectree_model_count.predict(X_test_count)

#Doğruluk
dectree_acc_count = metrics.accuracy_score(y_test, dectree_y_pred_count)
print("Accuracy:",dectree_acc_count)

Accuracy: 0.9962138084632517


### TF-IDF

In [15]:
#Modeli Eğitme
dectree_model_tfidf = dectree.fit(X_train_tf_idf,y_train)

#Tahmin
dectree_y_pred_tfidf = dectree_model_tfidf.predict(X_test_tf_idf)

#Doğruluk
dectree_acc_tfidf = metrics.accuracy_score(y_test, dectree_y_pred_tfidf)
print("Accuracy:",dectree_acc_tfidf)

Accuracy: 0.995916852264291


# Lojistik Regresyon

### Count Vectors 

In [16]:
from sklearn.linear_model import LogisticRegression

#Lojistik Regresyon Objesi
logreg = LogisticRegression()

#Modeli Eğitme
logreg_model_count = logreg.fit(X_train_count,y_train)

#Tahmin
logreg_y_pred_count = logreg_model_count.predict(X_test_count)

#Doğruluk
logreg_acc_count = metrics.accuracy_score(y_test, logreg_y_pred_count)
print("Accuracy:",logreg_acc_count)

Accuracy: 0.9958426132145508


D:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


### TF-IDF

In [17]:
#Modeli Eğitme
logreg_model_tfidf = logreg.fit(X_train_tf_idf,y_train)

#Tahmin
logreg_y_pred_tfidf = logreg_model_tfidf.predict(X_test_tf_idf)

#Doğruluk
logreg_acc_tfidf = metrics.accuracy_score(y_test, logreg_y_pred_tfidf)
print("Accuracy:",logreg_acc_tfidf)

Accuracy: 0.9859688195991091


# K-NN

### Count Vectors

In [18]:
from sklearn.neighbors import KNeighborsClassifier

#KNN Objesi
knn = KNeighborsClassifier()

#Modeli Eğitme
knn_model_count = knn.fit(X_train_count,y_train)

#Tahmin
knn_y_pred_count = knn_model_count.predict(X_test_count)

#Doğruluk
knn_acc_count = metrics.accuracy_score(y_test, knn_y_pred_count)
print("Accuracy:", knn_acc_count)

Accuracy: 0.781365998515219


### TF-IDF

In [19]:
#Modeli Eğitme
knn_model_tfidf = knn.fit(X_train_tf_idf,y_train)

#Tahmin
knn_y_pred_tfidf = knn_model_tfidf.predict(X_test_tf_idf)

#Doğruluk
knn_acc_tfidf = metrics.accuracy_score(y_test, knn_y_pred_tfidf)
print("Accuracy:", knn_acc_tfidf)

Accuracy: 0.6443207126948775


# Random Forest

### Count Vectors

In [20]:
from sklearn.ensemble import RandomForestClassifier

#Karar Ağacı Objesi
ranfor = RandomForestClassifier()

#Modeli Eğitme
ranfor_model_count = ranfor.fit(X_train_count,y_train)

#Tahmin
ranfor_y_pred_count = ranfor_model_count.predict(X_test_count)

#Doğruluk
ranfor_acc_count = metrics.accuracy_score(y_test, ranfor_y_pred_count)
print("Accuracy:", ranfor_acc_count)

Accuracy: 0.9850779510022272


### TF-IDF

In [21]:
## Modeli Eğitme
ranfor_model_tfidf = ranfor.fit(X_train_tf_idf,y_train)

#Tahmin
ranfor_y_pred_tfidf = ranfor_model_tfidf.predict(X_test_tf_idf)

#Doğruluk
ranfor_acc_tfidf = metrics.accuracy_score(y_test, ranfor_y_pred_tfidf)
print("Accuracy:", ranfor_acc_tfidf)

Accuracy: 0.9882702301410542
